In [1]:
# Import from used modules

import inputs_model as im

from pyomo.environ import *
import pandas as pd


#### Criação do Modelo

In [2]:
# Model
model = ConcreteModel()

# Sets:                            
model.Nodes = RangeSet( 0, (im.total_nodes - 1) ) # Node Set
model.Functions = RangeSet( 0, (im.total_functions - 1) ) # Function Set
model.Demands = RangeSet( 0, (im.total_demands - 1)) # Demand Set


# Variables:

# Mapping the virtual flow to the physical flow - of each demand

model.w = Var(  model.Nodes, 
                model.Nodes, 
                model.Functions, 
                model.Functions, 
                model.Demands, 
                within = Binary, 
                initialize = 0)

# Mapping if demand 'd' used function 'k' on server 'i'
model.u = Var(  model.Nodes, 
                model.Functions, 
                model.Demands, 
                within = Binary, 
                initialize = 0)

# Mapping if function 'k' are allocated on server 'i'
model.x = Var(  model.Nodes, 
                model.Functions, 
                within = Binary, 
                initialize = 0)

# Mapping physical links that were used
model.y = Var(  model.Nodes,
                model.Nodes, 
                within = Binary,  
                initialize = 0)

# Mapping of the total demands that used each physical link.
model.number_flows = Var(   model.Nodes,    
                            model.Nodes,
                            within = NonNegativeIntegers, 
                            initialize = 0)

# Fitness Function
def obj_function(model):
    """
    Calcula o valor da função objetivo do modelo de otimização.

    Args:
        model: O modelo de otimização que contém as variáveis de decisão.

    Returns:
        float: O valor da função objetivo calculado pelo modelo de otimização.
    """
    
    total_flow = sum(   model.number_flows[i, j] 
                        for i in model.Nodes 
                        for j in model.Nodes 
                        )
    
    placement = sum(    model.x[i,k] 
                        for i in model.Nodes        
                        for k in model.Functions
                    )

    return total_flow + placement
                         
# Objective Function
model.objective = Objective(    rule = obj_function, 
                                sense = minimize 
                            ) 

#### Restrições


<!-- 

def flow_used(model, d, i, l):
    '''
    A demanda d só pode utilizar a função l, alocada no servidor i,
    se ela passar pelo servidor i no seu fluxo físico.    
    '''

    source = im.demands[d]['source']
    virtual_topology = im.demands[d]['service']['sfc'] 

    # i não é o meu nó de origem
    if (source != i): 

        # A demanda d só pode usar a função l no nó i se ... 
        permission_used = model.u[i,l,d] <= \
                            sum(  model.w[j,i,k,l,d] # ... se a demanda d saiu do nó j (onde estava usando a função k) para o nó i (onde vai usar a função l)
                                for j in model.Nodes 
                                for k in model.Functions 
                                if (im.physical_topology[j][i] == 1) & (virtual_topology[k][l] == 1)
                                )
    
    else:
        permission_used = Constraint.Skip

    return permission_used

model.C9 = Constraint(  model.Demands, 
                        model.Nodes, 
                        model.Functions, 
                        rule = flow_used)






-->

In [3]:
def set_source(model, d):
    '''
    A primeira função (k) que a demanda (d) utiliza está alocada no servidor (i) de origem.
    '''
    
    # Origem da demanda
    origem_node = im.demands[d]['source'] 
    # A demanda d usou a função source (0) no nó de origem
    set_source = model.u[   origem_node, 
                            0, 
                            d] == 1 
    
    return set_source

model.C1 = Constraint(  model.Demands, 
                        rule = set_source)

In [4]:
def set_destiny(model, d):
    '''
    A última função (k) que a demanda (d) utiliza está alocada no servidor (i) de destino.
    '''
    
    # Destino da demanda
    destini_node = im.demands[d]['destiny'] 
    # A demanda d usou a função destiny (7) no nó de destino
    set_destination = model.u[  destini_node,
                                7,
                                d] == 1

    
    return set_destination

model.C2 = Constraint(  model.Demands, 
                        rule = set_destiny)

In [5]:
def set_used(model, d, k):
    '''
    A demanda d precisa utilizar todas as funções que exitem na sua cadeia.
    A variável u vai ser igual a 1 se a demanda d estiver usando a função k no servidor i.
    A demanda obrigatóriamente precisa usar todas suas funções.
    '''
    
    functions_used = im.demands[d]['service']['functions'][k]

    if functions_used == 1:
        # Se k é uma função que d precisa usar
        used = sum( model.u[i, k, d] # a demanda d pracisa usar a funcao k em algum nó i (seja ele qual for)
                    for i in model.Nodes) == 1
    else:
        used = Constraint.Skip

    return used

model.C3 = Constraint(  model.Demands, 
                        model.Functions, 
                        rule = set_used)

In [6]:
def function_allocation(model, d, i, k):
    '''
    A demanda d só consegue utilizar a função k no nó i se ...
    Se a função k estiver alocada no nó i
    '''
    

    return model.u[i, k, d] <= model.x[i, k]
    
model.C4 = Constraint(  model.Demands, 
                        model.Nodes, 
                        model.Functions, 
                        rule = function_allocation)

In [7]:
def flow_conservation(model, d, i, k, l):
    '''
    Conservação de Fluxo - Se existe um fluxo virtual, esse fluxo precisa ser mapeado para o fluxo fisico.
    Para toda demanda d que está no servidor i usando a função k - precisa ser direcionado para
    servidor j que vá usar a função l. k,l é o fluxo virtual que representa que ao sair da função k a demanda d
    precisa usar a função l. 
    '''


    virtual_topology = im.demands[d]['service']['sfc'][k][l]

    if (virtual_topology == 1):

        main_conservation = sum(    model.w[i,j,k,l,d] # A demanda d que está no nó i utilizando a função k - vai para o nó j utilizar a função l
                                    # (saindo do i)
                                    for j in model.Nodes 
                                    if ( im.physical_topology[i][j] == 1) ) - \
                                    \
                            sum(    model.w[j,i,k,l,d] # A demanda d que estava no nó j utilizando a função k - foi para o nó i utilizar a função l
                                    # (Entrando no I)
                                    for j in model.Nodes 
                                    if ( im.physical_topology[j][i] == 1) ) \
                            == model.u[i,k,d] - model.u[i,l,d] # A demanda d usou a função k no nó i  -  A demanda d usou a função l no nó i
    else:
        main_conservation = Constraint.Skip

        
    return main_conservation

model.C5 = Constraint(  model.Demands, 
                        model.Nodes, 
                        model.Functions, 
                        model.Functions, 
                        rule = flow_conservation)

In [8]:
def loop_in(model, j, d):
    '''
    Para evitar que uma demanda d fique em loop de entrada em um servidor i.
    Ou seja, para todos fluxos virtuais (k,l) da demanda d --> A demanda só pode 
    entrar no servido i uma única vez.
    '''

    virtual_topology = im.demands[d]['service']['sfc']

    avoid_loops_in = \
        sum(    model.w[i,j,k,l,d] # a demanda d está entrando no nó j (onde vai usar a funcao l) vindo do nó i (onde estava usando a funcao k) 
                for i in model.Nodes 
                for k in model.Functions 
                for l in model.Functions 
                if (im.physical_topology[i][j] == 1) & (virtual_topology[k][l] == 1)) <= 1

    return avoid_loops_in

model.C6 = Constraint(  model.Nodes, 
                        model.Demands, 
                        rule = loop_in)

In [9]:
def loop_out(model, i, d):
    '''
    Para evitar que uma demanda d fique em loop de saída de um servidor i.
    Ou seja, para todos fluxos virtuais (k,l) da demanda d --> A demanda só 
    pode sair do servido i uma única vez.
    '''

    virtual_topology = im.demands[d]['service']['sfc']

    avoid_loops_out = \
        sum(    model.w[i,j,k,l,d] # a demanda d está saindo do nó i (onde estava usando a funcao k) para o nó j (usar a funcao l)
                for j in model.Nodes 
                for k in model.Functions 
                for l in model.Functions 
                if (im.physical_topology[i][j] == 1) & (virtual_topology[k][l] == 1)) <= 1

    return avoid_loops_out 

model.C7 = Constraint(  model.Nodes, 
                        model.Demands, 
                        rule = loop_out)

In [10]:
def total_flow(model, i, j):
    '''
    number_flows é uma variável de controle que apresenta o número de demandas que estão utilizando o link físico i,j
    '''
    
    flow = model.number_flows[i,j] == sum(  model.w[i,j,k,l,d] 
                                            for k in model.Functions 
                                            for l in model.Functions 
                                            for d in model.Demands
                                            )
                
    return flow

model.C8 = Constraint(  model.Nodes, 
                        model.Nodes, 
                        rule = total_flow)

#### Solução

In [11]:
# Rodando o solver
solver = SolverFactory('cplex')
results = solver.solve( model, )

#### Outputs - Exibição

In [12]:
# Cria uma lista com os rotulos dos nodes
nodes = []
for i in model.Nodes:
    nodes.append(f'Node {i}')

# Cria uma lista com os rotulos das demandas
    demands = []
for d in model.Demands:
    demands.append(f'Demanda {d}')

funcoes = ['Source', 'NAT', 'FW', 'TM', 'WOC', 'VOC', 'IDS', 'Destiny']

# DataFrame de Nodes x Funções
df_placement = pd.DataFrame(columns=funcoes, index=nodes)

index = pd.MultiIndex.from_product([demands, nodes], names=['Demanda', 'Servidor'])
# Dataframe de Demanda-Nodes X Funções
df_used = pd.DataFrame(columns=funcoes, index=index)
# Dataframe de Demanda-Nodes X Nodes
df_percurso = pd.DataFrame(columns=nodes, index=index)

In [26]:
tf = sum(value(model.number_flows[:, :]))
p = sum(value(model.x[:, :])) - sum(value(model.x[:, 0])) - sum(value(model.x[:, 7])) 
obj = value(model.objective)


txt = f' A função objetivo é : {obj}  --> Fluxo ({tf}) + Alocação ({p})' 
print(txt)

 A função objetivo é : 26.0  --> Fluxo (12.0) + Alocação (6.0)


In [30]:
print(f"Funções x Servidores - Onde a função k está sendo alocada? (Variável : X)")
print(f"=" * 60)

for i in model.Nodes:
    n = f'Node {i}'
    for k in model.Functions:
        if value(model.x[i,k]) == 1:
            df_placement.loc[n, funcoes[k]] = value(model.x[i,k])
        else:
            df_placement.loc[n, funcoes[k]] = '.'

df_placement

Funções x Servidores - Onde a função k está sendo alocada? (Variável : X)


,Source,NAT,FW,TM,WOC,VOC,IDS,Destiny
Node 0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Node 1,1.0,.,.,.,.,.,.,1.0
Node 2,1.0,.,.,.,.,.,.,1.0
Node 3,1.0,.,.,.,.,.,.,1.0


In [15]:
print(f"Onde a demanda d está usando a função k?  (Variável : U)")
print(f"=" * 45)


for d in model.Demands:
    a = f'Demanda {d}'
    for i in model.Nodes:
        n = f'Node {i}'
        for k in model.Functions:
            if value(model.u[i,k,d]) == 1:
                df_used.loc[(a, n), funcoes[k]] = value(model.u[i,k,d])
            else:
                df_used.loc[(a, n), funcoes[k]] = '.'

df_used

Onde a demanda d está usando a função k?  (Variável : U)


Source  NAT   FW   TM  WOC  VOC  IDS Destiny
Demanda   Servidor                                             
Demanda 0 Node 0      1.0  1.0  1.0  1.0  1.0    .  1.0       .
          Node 1        .    .    .    .    .    .    .     1.0
          Node 2        .    .    .    .    .    .    .       .
          Node 3        .    .    .    .    .    .    .       .
Demanda 1 Node 0        .  1.0  1.0  1.0  1.0  1.0  1.0       .
          Node 1      1.0    .    .    .    .    .    .       .
          Node 2        .    .    .    .    .    .    .     1.0
          Node 3        .    .    .    .    .    .    .       .
Demanda 2 Node 0        .  1.0  1.0    .  1.0  1.0  1.0       .
          Node 1        .    .    .    .    .    .    .       .
          Node 2      1.0    .    .    .    .    .    .       .
          Node 3        .    .    .    .    .    .    .     1.0
Demanda 3 Node 0        .  1.0  1.0  1.0    .    .    .     1.0
          Node 1        .    .    .    .    .    .    .       .
          Node 2        .    .    .    .    .    .    .       .
          Node 3      1.0    .    .    .    .    .    .       .
Demanda 4 Node 0      1.0  1.0  1.0  1.0  1.0    .  1.0       .
          Node 1        .    .    .    .    .    .    .     1.0
          Node 2        .    .    .    .    .    .    .       .
          Node 3        .    .    .    .    .    .    .       .
Demanda 5 Node 0      1.0  1.0  1.0    .  1.0  1.0  1.0       .
          Node 1        .    .    .    .    .    .    .     1.0
          Node 2        .    .    .    .    .    .    .       .
          Node 3        .    .    .    .    .    .    .       .
Demanda 6 Node 0      1.0  1.0  1.0  1.0  1.0    .  1.0       .
          Node 1        .    .    .    .    .    .    .     1.0
          Node 2        .    .    .    .    .    .    .       .
          Node 3        .    .    .    .    .    .    .       .
Demanda 7 Node 0      1.0  1.0  1.0  1.0  1.0  1.0  1.0       .
          Node 1        .    .    .    .    .    .    .     1.0
          Node 2        .    .    .    .    .    .    .       .
          Node 3        .    .    .    .    .    .    .       .
Demanda 8 Node 0      1.0  1.0  1.0  1.0  1.0    .  1.0       .
          Node 1        .    .    .    .    .    .    .     1.0
          Node 2        .    .    .    .    .    .    .       .
          Node 3        .    .    .    .    .    .    .       .
Demanda 9 Node 0      1.0  1.0  1.0  1.0  1.0  1.0  1.0       .
          Node 1        .    .    .    .    .    .    .     1.0
          Node 2        .    .    .    .    .    .    .       .
          Node 3        .    .    .    .    .    .    .       .

In [16]:
print(f"Qual o fluxo da demanda d?  (Variável : W)")
print(f"=" * 30)

# Para cada demanda
for d in model.Demands:
    a = f'Demanda {d}'
    # Para cada nó 
    for i in model.Nodes:
        n1 = f'Node {i}'
        # Para cada nó
        for j in model.Nodes:
            n2 = f'Node {j}'

            # A demanda d utilizou o link j,i
            total = sum(value(model.w[i,j,:,:,d]))
            

            # ---------------------------------------------------------------------------
            if  total > 0:
                
                df_percurso.loc[(a, n1), n2] = sum(value(model.w[i,j,:,:,d]))
            else:
                df_percurso.loc[(a, n1), n2] = '.'

            # ---------------------------------------------------------------------------
            if (i == im.demands[d]['source']) & (j == im.demands[d]['source']):
                if total > 0:
                    df_percurso.loc[(a, n1), n2] = 'O1'
                else:
                    df_percurso.loc[(a, n1), n2] = 'O'

            # ---------------------------------------------------------------------------
            if (i == im.demands[d]['destiny']) & (j == im.demands[d]['destiny']):
                if total > 0:
                    df_percurso.loc[(a, n1), n2] = 'D1'
                else:
                    df_percurso.loc[(a, n1), n2] = 'D'

df_percurso

Qual o fluxo da demanda d?  (Variável : W)


Node 0 Node 1 Node 2 Node 3
Demanda   Servidor                            
Demanda 0 Node 0        O    1.0      .      .
          Node 1        .      D      .      .
          Node 2        .      .      .      .
          Node 3        .      .      .      .
Demanda 1 Node 0        .      .    1.0      .
          Node 1      1.0      O      .      .
          Node 2        .      .      D      .
          Node 3        .      .      .      .
Demanda 2 Node 0        .      .      .    1.0
          Node 1        .      .      .      .
          Node 2      1.0      .      O      .
          Node 3        .      .      .      D
Demanda 3 Node 0        D      .      .      .
          Node 1        .      .      .      .
          Node 2        .      .      .      .
          Node 3      1.0      .      .      O
Demanda 4 Node 0        O    1.0      .      .
          Node 1        .      D      .      .
          Node 2        .      .      .      .
          Node 3        .      .      .      .
Demanda 5 Node 0        O    1.0      .      .
          Node 1        .      D      .      .
          Node 2        .      .      .      .
          Node 3        .      .      .      .
Demanda 6 Node 0        O    1.0      .      .
          Node 1        .      D      .      .
          Node 2        .      .      .      .
          Node 3        .      .      .      .
Demanda 7 Node 0        O    1.0      .      .
          Node 1        .      D      .      .
          Node 2        .      .      .      .
          Node 3        .      .      .      .
Demanda 8 Node 0        O    1.0      .      .
          Node 1        .      D      .      .
          Node 2        .      .      .      .
          Node 3        .      .      .      .
Demanda 9 Node 0        O    1.0      .      .
          Node 1        .      D      .      .
          Node 2        .      .      .      .
          Node 3        .      .      .      .